In [ ]:
import os
import pandas as pd
from openai import OpenAI

from dotenv import load_dotenv
load_dotenv("../keys.env")
api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
df6 = pd.read_csv("./data/token.csv")
df6['len'] = df6['contents'].str.len()

In [ ]:
df6

In [ ]:
llm = OpenAI(api_key=api_key)

In [ ]:
def generate_training_data(corpus, openai):
    evolving = f"나는 너가 질문 작성자로 활동하기를 원해. 너의 목표는 주어진 문단을 활용하여 AI가 학습할 질문을 만드는거야. 질문은 합리적이어야 하고, 일상에서 나올 수 있는 질문이어야하며, 인간이 이해하고 응답할 수 있어야 해.\n\
        그리고 답변은, 근거를 들어서 논리적으로 답변을 해야해. 또한,질문과 답변을 다섯개를 만들어줘.\n\
        최대한 질문이 장황해지지 않도록 노력해야 하며, 추론이나 일상생활에서 생길 수 있는 문제들을 담아야해. 무조건 한국어로 작성해줘. 생성된 질문에는 질문이나 답변에 숫자 없이 '#질문:'이라는 단어로 시작해야하고, 생성된 답변에는 '#답변:'이라는 단어로 시작해야해.\n\
        주어진 문단: {corpus}\n\n"
    
    chat_completion = openai.chat.completions.create(
        model='gpt-4o',
        messages=[{"role":"user", "content": evolving}],
        temperature=0.7,
        max_tokens=8192)
    
    return chat_completion.choices[0].message.content

In [ ]:
example1 = generate_training_data(df6['contents'][1], llm)

In [ ]:
example1

In [ ]:
len(example1.lstrip().split('#'))

In [ ]:
## 반환값에 "질문"과 "답변"을 제거함.

def remove_str(tmp_str):
    if '질문:' in tmp_str:
        tmp_str= tmp_str[3:]
        tmp_str=tmp_str.lstrip()
        tmp_str=tmp_str.rstrip()
    elif '답변:' in tmp_str:
        tmp_str= tmp_str[3:]
        tmp_str=tmp_str.lstrip()
        tmp_str=tmp_str.rstrip()
    elif '질문1:' in tmp_str:
        tmp_str= tmp_str[4:]
        tmp_str=tmp_str.lstrip()
        tmp_str=tmp_str.rstrip()
    elif '답변1:' in tmp_str:
        tmp_str= tmp_str[4:]
        tmp_str=tmp_str.lstrip()
        tmp_str=tmp_str.rstrip()
    elif '질문2:' in tmp_str:
        tmp_str= tmp_str[4:]
        tmp_str=tmp_str.lstrip()
        tmp_str=tmp_str.rstrip()
    elif '답변2:' in tmp_str:
        tmp_str= tmp_str[4:]
        tmp_str=tmp_str.lstrip()
        tmp_str=tmp_str.rstrip()
    elif '질문3:' in tmp_str:
        tmp_str= tmp_str[4:]
        tmp_str=tmp_str.lstrip()
        tmp_str=tmp_str.rstrip()
    elif '답변3:' in tmp_str:
        tmp_str= tmp_str[4:]
        tmp_str=tmp_str.lstrip()
        tmp_str=tmp_str.rstrip()
    elif '질문4:' in tmp_str:
        tmp_str= tmp_str[4:]
        tmp_str=tmp_str.lstrip()
        tmp_str=tmp_str.rstrip()
    elif '답변4:' in tmp_str:
        tmp_str= tmp_str[4:]
        tmp_str=tmp_str.lstrip()
        tmp_str=tmp_str.rstrip()
    elif '질문5:' in tmp_str:
        tmp_str= tmp_str[4:]
        tmp_str=tmp_str.lstrip()
        tmp_str=tmp_str.rstrip()
    elif '답변5:' in tmp_str:
        tmp_str= tmp_str[4:]
        tmp_str=tmp_str.lstrip()
        tmp_str=tmp_str.rstrip()
    else: 
        print('############## warning!!!!!!!!!!! ##############')
        print(tmp_str)
        print('#################################################')
        tmp_str=False

    return tmp_str

In [ ]:
from tqdm import tqdm 

q_list=[]
a_list=[]

for i in tqdm(range(len(df6))):
    tmp=True
    chunk=df6['contents'][i]

    while tmp:
        res= generate_training_data(chunk, llm)

        res_lst=res.lstrip().split('#')

        if (len(res_lst)==11) and '질문:' in res_lst[1] and '답변:' in res_lst[2] and '질문:' in res_lst[3] and '답변:' in res_lst[4] and '질문:' in res_lst[5] and '답변:' in res_lst[6] and \
        '질문:' in res_lst[7] and '답변:' in res_lst[8] and '질문:' in res_lst[9] and '답변:' in res_lst[10]: 
            q1 = remove_str(res_lst[1])
            q2 = remove_str(res_lst[3])
            q3 = remove_str(res_lst[5])
            q4 = remove_str(res_lst[7])
            q5 = remove_str(res_lst[9])

            a1 = remove_str(res_lst[2])
            a2 = remove_str(res_lst[4])
            a3 = remove_str(res_lst[6])
            a4 = remove_str(res_lst[8])
            a5 = remove_str(res_lst[10])

            if q1!=False and q2!=False and q3!=False and q4!=False and q5!=False and a1!=False and a2!=False and a3!=False and a4!=False and a5!=False:
                tmp=False
                q_list+=[q1,q2,q3,q4,q5]
                a_list+=[a1,a2,a3,a4,a5]
            else:
                print('something wrong!!!')

        else:
            print(res)


In [ ]:
train_df= pd.DataFrame({'question':q_list,'response':a_list})
train_df.to_excel('./data/insurance_train.xlsx', index=False)

In [ ]:
## chunk에 대한 정보를 evolving(증강)을 위해 저장함.

cor_lst=[]

for i in range(len(df6)):
    cor_tmp = df6['contents'][i]
    cor_lst+=[cor_tmp, cor_tmp, cor_tmp, cor_tmp, cor_tmp]

In [ ]:
train_df['corpus']=cor_lst

In [ ]:
## 질문과 응답으로 구성된 데이터가 구축되었다.
train_df.to_excel('./data/insurance_train_corpus.xlsx', index=False)

In [ ]:
eval_df= pd.read_csv('token_short.csv')

In [ ]:
eval_df

In [ ]:
llm = OpenAI(api_key=api_key)

def generate_eval_multi(instruction, openai):
    evolving= f"나는 너가 질문 작성자로 활동하기를 원해. 너의 목표는 주어진 문단을 활용하여 AI를 평가하는 질문을 만드는거야. 질문은 합리적이어야 하고, 모델의 성능을 해당 문단기반으로 잘 이해하고 있는지 평가할 수 있어야해.\n\
        또한, 4개의 선택지가 있는 multiple choice 문제를 생성하는거야.각각 선택지 번호는 'a.','b.','c.','d.'로 나와야 하며, 답변에는 해당선택지를 그대로 작성해야 해야하고 무작위 번호여야해.\n\
        최대한 질문이 장황해지지 않도록 노력해야 하며, 추론이나 일상생활에서 생길 수 있는 문제들을 담아야해. 무조건 한국어로 두개만 작성해줘. 생성된 질문에는 질문이나 답변에 숫자 없이 '#질문:'이라는 단어로 시작해야하고, 생성된 답변에는 '#답변:'이라는 단어로 시작해야해.\n\
        주어진 문단: {instruction}\n\n"
    
    chat_completion = openai.chat.completions.create(
        model='gpt-4o',
        messages=[{"role":"user", "content": evolving}],
        temperature=0.7,
        max_tokens=8192)
    
    return chat_completion.choices[0].message.content

In [ ]:
ex=generate_eval_multi(eval_df['contents'][1], llm)

In [ ]:
print(ex)

In [ ]:
len(ex.split('#'))

In [ ]:
ex.split('#')

In [ ]:
def check_multi_a(instruct):
    if 'a.' in instruct or 'b.' in instruct or 'c.' in instruct or 'd.' in instruct:
        return True
    else:
        return False

In [ ]:
q_list=[]
a_list=[]


for i in tqdm(range(len(eval_df))):
    tmp=True
    cor= eval_df['contents'][i]
    while tmp:
        res = generate_eval_multi(cor, llm)

        res = res.replace('(a)', 'a.')
        res = res.replace('(b)', 'b.')
        res = res.replace('(c)', 'c.')
        res = res.replace('(d)', 'd.')


        res = res.replace('a)', 'a.')
        res = res.replace('b)', 'b.')
        res = res.replace('c)', 'c.')
        res = res.replace('d)', 'd.')


        res_lst = res.split('#')


        if len(res_lst)==5 and '질문' in res_lst[1] and '답변' in res_lst[2] and '질문' in res_lst[3] and '답변' in res_lst[4] and check_multi_a(res_lst[4]) and check_multi_a(res_lst[2]):

            q1 = remove_str(res_lst[1].lstrip().rstrip())
            q2 = remove_str(res_lst[3].lstrip().rstrip())
            a1 = remove_str(res_lst[2].lstrip().rstrip())
            a2 = remove_str(res_lst[4].lstrip().rstrip())
            q_list += [q1,q2]
            a_list += [a1,a2]
            tmp = False

        else:
            print(res)
            

In [ ]:
few_shot_df= pd.DataFrame({'question':q_list,'answer': a_list})
few_shot_df.to_excel('./data/few_shot_eval.xlsx',index=False)